In [2]:
import torch
import os
import torchvision
from PIL import Image
from torch.utils.data import DataLoader
from torchvision.models.detection import fasterrcnn_resnet50_fpn
import torchvision.transforms as T

In [ ]:
# CHECKING DATASET STRUCTURE

train_path="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Data/train"
eval_path="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Data/eval"
test_path="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Data/test"

train_folder=os.listdir(train_path)
eval_folder=os.listdir(eval_path)
test_folder=os.listdir(test_path)

print(f"Nmumber of training samples: {len(train_folder)}")
print(f"Nmumber of evaluation samples: {len(eval_folder)}")
print(f"Nmumber of testing samples: {len(test_folder)}")
print(f"Total number of samples: {len(train_folder)+len(test_folder)+len(eval_folder)}")

# CHECKING ERRORS FOR FUNCTION: get_bounding_box

for file in train_folder: # executed also for eval and test set with no Error raised
    numbers=file.split("-")
    values=numbers[3]

    # if len(values) != 31:                                                                   questo controllo non serve in quanto dipende da quante cifre ogni numero ha
    #     raise ValueError(f" 1 st exception: Invalid plate text format in filename: {file}")
    
    values_v2=values.split("&")

    if len(values_v2) != 5:
        raise ValueError(f"2 nd exception: Invalid plate text format in filename: {file}")
    
    values_v3=[]
    for i in range(len(values_v2)):
        if "_" in values_v2[i]:
            values_v3.append(values_v2[i].split("_"))

    if len(values_v3) != 3:
        raise ValueError(f"3 rd exception: Invalid plate text format in filename: {file}")
    
    t=[values_v2[0],values_v3[0],values_v3[1],values_v3[2],values_v2[-1]]
    final_values = [int(x) for item in t for x in (item if isinstance(item, list) else [item])]
    x_coords=[final_values[0],final_values[2],final_values[4],final_values[6]]
    y_coords=[final_values[1],final_values[3],final_values[5],final_values[7]]

    if len(x_coords) != 4:
        raise ValueError(f"4 th exception: Invalid plate text format in filename: {file}")

 # CHECKING ERRORS FOR FUNCTION: get_text
    
provinces = ["皖", "沪", "津", "渝", "冀", "晋", "蒙", "辽", "吉", "黑", "苏", "浙", "京", "闽", "赣", "鲁", "豫", "鄂", "湘", "粤", "桂", "琼", "川", "贵", "云", "藏", "陕", "甘", "青", "宁", "新", "警", "学", "O"]
alphabet = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'O']
ads = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'O']

for file in train_folder: # executed also for eval and test set with no Error raised
        values=file.split("-")

        if len(values)!=7:
            raise ValueError(f"1 st exception occurred: Invalid plate text format in filename: {file}")
        
        text=str(values[4])

        # if len(text)!=7:                                                                                questo controllo non serve in quanto dipende da quante cifre ogni numero ha cifre diverse
        #     raise ValueError(f"2 nd exception occurred: Invalid plate text format in filename: {file}") 
        
        indices=text.split("_")

        if len(indices) != 8:
            raise ValueError(f"3 rd exception occurred: Invalid plate text format in filename: {file}")
        
        province_character=provinces[int(indices[0])]
        alphabet_character=alphabet[int(indices[1])]
        ads_charachters=[ads[int(i)] for i in indices[2:]]
        plate_text=province_character+alphabet_character+"".join(ads_charachters)

        if len(plate_text) != 8:
            raise ValueError(f"4 th exception occurred: Invalid plate text format in filename: {file}")

# ESEMPIO DI PROVA CON DUE FILE PER FUNZIONE: get_bounding_box

file="308069444444444444-91_94-8&424_589&564-578&564_27&553_8&431_589&424-0_0_3_24_32_25_32_25-100-241.jpg" # FILE CHE NON DA PROBLEMI CON LA LUNGHEZZA DI VALUES
numbers=file.split("-")
values=numbers[3]
values_v2=values.split("&")
values_v3=[]
for i in range(len(values_v2)):
    if "_" in values_v2[i]:
        values_v3.append(values_v2[i].split("_"))
t=[values_v2[0],values_v3[0],values_v3[1],values_v3[2],values_v2[-1]]
final_values = [int(x) for item in t for x in (item if isinstance(item, list) else [item])]
x_coords=[final_values[0],final_values[2],final_values[4],final_values[6]]
y_coords=[final_values[1],final_values[3],final_values[5],final_values[7]]
x_min = min(x_coords)
y_min = min(y_coords)
x_max = max(x_coords)
y_max = max(y_coords)

file2="0115711805556-88_269-312&388_484&456-484&445_314&456_312&396_481&388-0_0_3_24_28_28_28_28-154-41.jpg" # FILE CHE DA PROBLEMI CON LA LUNGHEZZA DI VALUES
numbers=file.split("-")
values=numbers[3]
values_v2=values.split("&")
values_v3=[]
for i in range(len(values_v2)):
    if "_" in values_v2[i]:
        values_v3.append(values_v2[i].split("_"))
t=[values_v2[0],values_v3[0],values_v3[1],values_v3[2],values_v2[-1]]
final_values = [int(x) for item in t for x in (item if isinstance(item, list) else [item])]
x_coords=[final_values[0],final_values[2],final_values[4],final_values[6]]
y_coords=[final_values[1],final_values[3],final_values[5],final_values[7]]
x_min = min(x_coords)
y_min = min(y_coords)
x_max = max(x_coords)
y_max = max(y_coords)

# ESEMPIO DI PROVA CON DUE FILE PER FUNZIONE: get_text

file="30237890625-87_88-197&472_449&567-449&553_201&567_197&472_448&477-10_2_3_24_31_33_26_24-192-75.jpg" # FILE CHE NON DA PROBLEMI CON LA LUNGHEZZA DI TEXT
values=file.split("-")
text=str(values[4])
indices=text.split("_")
if len(indices) != 8:
    raise ValueError(f"Invalid plate text format in filename: {file}")
province_character=provinces[int(indices[0])]
alphabet_character=alphabet[int(indices[1])]
ads_charachters=[ads[int(i)] for i in indices[2:]]
plate_text=province_character+alphabet_character+"".join(ads_charachters)
print(values,text,len(text),indices,plate_text)

file2="0115711805556-88_269-312&388_484&456-484&445_314&456_312&396_481&388-0_0_3_24_28_28_28_28-154-41.jpg" # FILE CHE DA PROBLEMI CON LA LUNGHEZZA DI TEXT
values=file2.split("-")
text=str(values[4])
indices=text.split("_")
if len(indices) != 8:
    raise ValueError(f"Invalid plate text format in filename: {file}")
province_character=provinces[int(indices[0])]
alphabet_character=alphabet[int(indices[1])]
ads_charachters=[ads[int(i)] for i in indices[2:]]
plate_text=province_character+alphabet_character+"".join(ads_charachters)
print(values,text,len(text),indices,plate_text)

In [9]:
# NUOVO DATASET CON PIU ELEMENTI DI TRAIN E MENO DI TEST

path_train_og="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Data/train"
path_test_og="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Data/test"
path_train_tf="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Data_transf/train"
path_test_tf="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Data_transf/test"
folder_train_og=os.listdir(path_train_og)
folder_test_og=os.listdir(path_test_og)
folder_train_tf=os.listdir(path_train_tf)
folder_test_tf=os.listdir(path_test_tf)
print(f"number of original training samples: {len(folder_train_og)}")
print(f"number of original test samples: {len(folder_test_og)}")
print(f"number of new training samples: {len(folder_train_tf)}")
print(f"number of new training samples: {len(folder_test_tf)}")

number of original training samples: 5291
number of original test samples: 3329
number of new training samples: 6635
number of new training samples: 1985


In [1]:
# BOUNDING BOX FUNCTION 

def get_bounding_box(file):
    numbers=file.split("-")
    values=numbers[3]
    values_v2=values.split("&")
    values_v3=[]
    for i in range(len(values_v2)):
        if "_" in values_v2[i]:
            values_v3.append(values_v2[i].split("_"))
    t=[values_v2[0],values_v3[0],values_v3[1],values_v3[2],values_v2[-1]]
    final_values = [int(x) for item in t for x in (item if isinstance(item, list) else [item])]
    x_coords=[final_values[0],final_values[2],final_values[4],final_values[6]]
    y_coords=[final_values[1],final_values[3],final_values[5],final_values[7]]
    x_min = min(x_coords)
    y_min = min(y_coords)
    x_max = max(x_coords)
    y_max = max(y_coords)
    
    return [float(x_min), float(y_min), float(x_max), float(y_max)]

In [3]:
# INTERSECTION OVER UNION FUNCTION

def compute_IoU(box1, box2):

    box1=box1.squeeze()
    box2=box2.squeeze()

    xA = max(box1[0], box2[0])
    yA = max(box1[1], box2[1])
    xB = min(box1[2], box2[2])
    yB = min(box1[3], box2[3])

    area_of_intersection = max(0, xB - xA) * max(0, yB - yA)

    area_box1 = (box1[2] - box1[0]) * (box1[3]- box1[1])
    area_box2 = (box2[2] - box2[0]) * (box2[3] - box2[1])

    IoU = area_of_intersection / float(area_box1 + area_box2 - area_of_intersection)

    return IoU

In [ ]:
# DATASET   

import os
import torch
from PIL import Image
from torchvision.transforms import transforms
from torch.utils.data import DataLoader

smaller_train_path="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Prove/train-20250508T142228Z-1-001/train"
folder=os.listdir(smaller_train_path)

class CCPD_Dataset(torch.utils.data.Dataset):
    
    def __init__(self, path, transforms=None):
        self.path = path
        self.transforms = transforms
        self.folder = os.listdir(path)
        self.images = [f for f in self.folder if f.endswith(".jpg")]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        file = self.images[idx]
        full_path = os.path.join(self.path, file)
        image = Image.open(full_path).convert("RGB")

        bbox = get_bounding_box(file)
        box = torch.tensor([bbox], dtype=torch.float32)
        label = torch.tensor([1], dtype=torch.int64)  

        target = {"boxes": box, "labels": label}

        if target["boxes"].shape[0] != target["labels"].shape[0]:
            raise ValueError(f"Mismatch in number of boxes and labels for file: {file}")

        if self.transforms:
            image = self.transforms(image)

        return image, target
    
def collate_fn(batch):
    return tuple(zip(*batch))

train_path2="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Prove/train2"
eval_path2="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Prove/eval2"
test_path2="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Prove/test2"
    
transform = transforms.ToTensor()
train_dataset = CCPD_Dataset(train_path2, transforms=transform)
eval_dataset = CCPD_Dataset(eval_path2, transforms=transform)
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn, num_workers=0)
eval_dataloader = DataLoader(eval_dataset, batch_size=2, shuffle=False, collate_fn=collate_fn, num_workers=0)

In [ ]:
# MODEL

import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn
import torch.optim as optim

model = fasterrcnn_resnet50_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes=2)
# device = torch.device("cuda" if torch.cuda.is_available else "cpu")
# model.to(device)
device="cpu" 
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

model.train()
for epoch in range(5):
    model.train()
    train_loss = 0
    for images, targets in train_dataloader:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        train_loss += losses.item()
    print(f"Epoch {epoch} Training Loss: {train_loss:.4f}")

    model.eval()
    eval_loss = 0
    with torch.no_grad():
        total_iou = 0
        count = 0
        for images, targets in eval_dataloader:
            images = [img.to(device) for img in images]
            outputs = model(images)

            for pred, target in zip(outputs, targets):
                pred_boxes = pred['boxes'].to(device)
                gt_boxes = target['boxes'].to(device)

                if len(pred_boxes) > 0:
                    iou = compute_IoU(pred_boxes[0], gt_boxes[0])
                    total_iou += iou
                    count += 1

    avg_iou = total_iou / count if count > 0 else 0
    print(f"Average IoU on eval set: {avg_iou:.4f}")

In [ ]:
# SAVING THE MODEL

torch.save(model.state_dict(), "model_weights.pth")
model=fasterrcnn_resnet50_fpn()
model.load_state_dict(torch.load("model_weights.pth"))

In [ ]:
# EXAMPLE OF THE MODEL

from PIL import Image
import matplotlib.pyplot as plt
import torchvision.transforms as T
import torch

model.eval()
img_path = "/home/filippo/Documents/Visual Studio Code/Computer_Vision/Data/train/04-90_267-158&448_542&553-541&553_162&551_158&448_542&450-0_1_3_24_27_33_30_24-99-116.jpg"  # Replace with a valid image path
image = Image.open(img_path).convert("RGB")
transform = T.ToTensor()
img_tensor = transform(image).unsqueeze(0).to(device) 

with torch.no_grad():
    prediction = model(img_tensor) # PREDICTION IS A LIST OF DICTIONARIES; EACH DICTIONARY CONTAINS: "BOXES": LIST CONTAINING THE COORDINATES OF THE BOUNDING 
    # BOXES COMPUTED BY THE MODEL AND "SCORES":  LIST CONTAINING THE CONFIDENCE VALUES OF ALL THE BOUNDING BOXES FOUND BY THE MODEL

boxes = prediction[0]['boxes'].cpu()
scores = prediction[0]['scores'].cpu()
threshold = 0.1
keep = scores >= threshold
boxes = boxes[keep]
plt.figure(figsize=(10, 8))
plt.imshow(image)
ax = plt.gca()
for box in boxes:
    x1, y1, x2, y2 = box
    rect = plt.Rectangle((x1, y1), x2 - x1, y2 - y1,
                         fill=False, edgecolor='red', linewidth=2)
    ax.add_patch(rect)
plt.title("Predicted Car Plate Bounding Box")
plt.axis("off")
plt.show()

In [6]:
# CROP FUNCTION WITH PREDICTED BOUNDING BOX

def crop_image_with_RCNN(file):
    
    image = Image.open(file).convert("RGB")
    transform = T.ToTensor()
    img_tensor = transform(image).unsqueeze(0).to(device) 
    with torch.no_grad():
        prediction = model(img_tensor)

    boxes = prediction[0]['boxes']
    scores = prediction[0]['scores']
    max_confidence=0
    idx=-1

    for i in range(len(scores)):
        if scores[i].item()>=max_confidence:
            max_confidence=scores[i].item()
            idx=i

    best_bb=boxes[idx]
    best_bb=best_bb.int()
    cropped_image = img_tensor[0, :, best_bb[1]:best_bb[3], best_bb[0]:best_bb[2]]

    return cropped_image

def crop_image_with_ground_truth(full_path):
    filename = os.path.basename(full_path)  # extract just the filename for parsing
    bb = get_bounding_box(filename)
    image = Image.open(full_path).convert("RGB")
    cropped_image = image.crop(bb)
    return cropped_image

def crop_folder(folder_path):
    cropped_folder = []
    files = os.listdir(folder_path)
    for file in files:
        full_path = os.path.join(folder_path, file)
        cropped_image = crop_image_with_ground_truth(full_path)
        cropped_folder.append(cropped_image)
    return cropped_folder

In [ ]:
# CROPPED IMAGE EXAMPLE

path="/home/filippo/Documents/Visual Studio Code/Computer_Vision/Prove/train-20250508T142228Z-1-001/train"
cropped_folder=crop_folder(path)
cropped_folder[0]